# Agents and Tools

In this section of the lab, we will review some agent concepts and see how agents can interact the basic tools

- Define a basic agent, using the `builtin::websearch` tool from Llama Stack
- Get familiar with concepts like "Turns" and "Steps" and "Tool_Calling" that the agent takes
- Run the agent to return web results
- Optional: experience a prompt chaining example

> **Note:**
>This section is intended only as an introduction to Agent concepts and language, and we will get into more exciting work in the next modules. If you are familiar with these concepts, feel free to skip this section.
> 


> **Note:**
> For the sake of simplicity and time, we will skip through some of the python basics we went through in previous modules and treat this more like a nomral python application 
> 

### Install Python Prerequisist

As always, let's start by installing the Python Libraries we neeed, we will add some additional libraries to support some of our tools. 

In [1]:
!pip install -U llama-stack-client==0.2.7 dotenv geocoder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [geocoder]1/3 [future]


### Define the LLamastack server and Model

Let's define the libraries we need and our basic Llama Stack configurations:

In [2]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from src.utils import step_printer
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'

# We will be using the Tavily web search service (docs.tavily.com/)
tavily_search_api_key='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'
provider_data = {"tavily_search_api_key": tavily_search_api_key}

> **Note:**
>When running this code in a regular Python application, we would usually like to read environment variables from an `.env` file, for our needs in this lab, we will hard code these in this cell, to make things more clear
>

### Initialize the *Client* 
We will not define our client, and introduce a new pattern/trick that will help us later when we try to move between models and other providers within Llama Stack. 

In [3]:
LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)
# List available models and select from allowed models list
allowed_models_list=["meta-llama/Llama-3.2-3B-Instruct"]
selected_model = None

models = client.models.list()

print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
# If no allowed model was found, you might want to handle that case
if selected_model is None:
    print("No allowed model found in the list.")


print(f"Selected model (from allowed list): {selected_model}")
            # Removed the break here to show all available models, but the selection logic remains picking the first one


SELECTED_MODEL = selected_model

--- Available models: ---
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
all-MiniLM-L6-v2 - ollama - all-minilm:latest
Selected model (from allowed list): meta-llama/Llama-3.2-3B-Instruct


### Define the a single tool agent
Here we define our new agent, we will pass along the `client` defining our connection to the Llama Stack server, the `model` we selected in the previous step, the `tools` array with all available tools for the agent, and crucually, the prompt, named `instructions` for the agent.

> **Note:**
> You might notice that we are being very specific in our prompt, for the purpose of lab we are using rather small models and are providing a narrow path to follow, in a real world scenario, prompts and LLMs can be experimented with to produce the best results.
> 

In [4]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
)

### inspecting the Agent's response structure

Let's give our agent a couple of easy questions to see how it calls our `builtin::websearch` tool.
Notice that we are generating a new session ID for each question as convesation memory is shared throughtout the session. (More on this later!)


In [6]:
user_prompts = [
       "search the web for the latest in OpenShift?",
       "search for the current weather in boston",
]


for prompt in user_prompts:
    # Generate a new Unique Identifier for each session 
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"web-session-{new_uuid}")

    cprint(f"\n{'='*100}\nProcessing user query: {prompt}\n{'='*100}")

    
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: search the web for the latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest news OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest news OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Azure Red Hat OpenShift: April 2025 Update | Microsoft Community Hub',
│   │   │   'url': 'https://techcommunity.microsoft.com/blog/appsonazureblog/azure-red-hat-openshift-april-2025-update/4400331',
│   │   │   'content': "OpenShift 4.16 on Day 0 → Now available for new ARO cluster installs with the latest features and improvements. See what's new. Note that a cluster version upgrade support (Day 2 operation) kicks in for existing clusters as soon as an upgrade path is available. This happens shortly after OCP GA availability for a version.",
│   │   │   'score': 0.769098,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Experience the many enhancements in OpenShift 4.18',
│   │   │   'url': 'https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418',
│   │   │   'content': 'Red Hat Enhances Security and Virtualization Experience with Latest Version of Red Hat OpenShift . Red Hat OpenShift Container Platform 4.18 release notes. Get started: Start your OpenShift journey with your Developer Sandbox. Find resources for Getting Started with OpenShift. Discover more ways to get started by downloading Red Hat OpenShift.',
│   │   │   'score': 0.7251239,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "Red Hat's new OpenShift delivers AI, edge and security enhancements",
│   │   │   'url': 'https://www.zdnet.com/article/red-hats-new-openshift-delivers-ai-edge-and-security-enhancements/',
│   │   │   'content': "OpenShift AI 2.15: Set to release later this month, it introduces a new model registry for managing AI models, data drift detection for maintaining model reliability, and integration with Nvidia's",
│   │   │   'score': 0.71137124,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Latest Enhancements and Features Introduced in the versions of Red Hat ...',
│   │   │   'url': 'https://access.redhat.com/articles/7048492',
│   │   │   'content': "What's new in Red Hat OpenShift Container Platform. Red Hat OpenShift is a Kubernetes-based app dev platform that enables developers to build, deploy, and manage containerized applications. It provides a comprehensive set of tools and services for developers, operations teams, and IT organizations to streamline application development and delivery.",
│   │   │   'score': 0.68420124,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's new in Red Hat OpenShift",
│   │   │   'url': 'https://www.redhat.com/en/whats-new-red-hat-openshift',
│   │   │   'content': 'Red Hat AI A portfolio for developing and deploying artificial intelligence solutions across the hybrid cloud.; Red Hat Enterprise Linux A flexible, stable operating system to support hybrid cloud innovation.; Red Hat OpenShift A container platform to build, modernize, and deploy applications at scale.; Red Hat Ansible Automation Platform A foundation for implementing enterprise-wide automation.',
│   │   │   'score': 0.42935818,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are the latest news and updates on OpenShift:

* Azure Red Hat OpenShift: April 2025 Update is now available for new ARO cluster installs with the latest features and improvements.
* Experience the many enhancements in OpenShift 4.18, which includes security and virtualization enhancements.
* Red Hat's new OpenShift delivers AI, edge, and security enhancements, including a new model registry for managing AI models and data drift detection for maintaining model reliability.
* Latest Enhancements and Features Introduced in the versions of Red Hat OpenShift Container Platform provide a comprehensive set of tools and services for developers, operations teams, and IT organizations to streamline application development and delivery.
* What's new in Red Hat OpenShift includes a portfolio for developing and deploying artificial intelligence solutions across the hybrid cloud, a flexible and stable operating system for hybrid 

{
│   'query': 'current weather in Boston',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Boston',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Boston', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.3583, 'lon': -71.0603, 'tz_id': 'America/New_York', 'localtime_epoch': 1747504074, 'localtime': '2025-05-17 13:47'}, 'current': {'last_updated_epoch': 1747503900, 'last_updated': '2025-05-17 13:45', 'temp_c': 20.6, 'temp_f': 69.1, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 200, 'wind_dir': 'SSW', 'pressure_mb': 1002.0, 'pressure_in': 29.58, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 81, 'cloud': 100, 'feelslike_c': 20.6, 'feelslike_f': 69.1, 'windchill_c': 21.3, 'windchill_f': 70.4, 'heatindex_c': 23.7, 'heatindex_f': 74.7, 'dewpoint_c': 18.4, 'dewpoint_f': 65.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 3.1, 'gust_mph': 6.7, 'gust_kph': 10.7}}",
│   │   │   'score': 0.9690604,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boston weather in May 2025 | Boston 14 day weather',
│   │   │   'url': 'https://www.weather25.com/north-america/usa/massachusetts/boston?page=month&month=May',
│   │   │   'content': "May 17. 0 mm. 22 ° / 16 ° Sunday. May 18. 0 ... Our weather forecast can give you a great sense of what weather to expect in Boston in May 2025. If you're planning to visit Boston in the near future, we highly recommend that you review the 14 day weather forecast for Boston before you arrive. Temperatures.",
│   │   │   'score': 0.9147261,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'May 17 Weather in Boston: Detailed Weather Forecast for May 17 in ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/boston/17-may/',
│   │   │   'content': 'Detailed ⚡ Weather Forecast for May 17 in Boston, Massachusetts, United States - 🌡️ temperature, wind, atmospheric pressure, humidity and precipitations - World-Weather.info. Add the current city. Search. Weather; Archive; Widgets °F. ... May 15 May 16 Select date: May 18 May 19. May 17, 2025 : Atmospheric conditions and temperature °F',
│   │   │   'score': 0.84277284,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Past Weather in Boston, Massachusetts, USA — May 2025 - timeanddate.com',
│   │   │   'url': 'https://www.timeanddate.com/weather/usa/boston/historic?month=5&year=2025',
│   │   │   'content': 'Weather reports from May 2025 in Boston, Massachusetts, USA with highs and lows',
│   │   │   'score': 0.6578248,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boston, MA Monthly Weather | AccuWeather',
│   │   │   'url': 'https://www.accuweather.com/en/us/boston/02108/may-weather/348735',
│   │   │   'content': 'Get the monthly weather forecast for Boston, MA, including daily high/low, historical averages, to help you plan ahead.',
│   │   │   'score': 0.1480598,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The current weather in Boston is overcast with a temperature of 20.6°C (69.1°F). The wind is blowing at 4.7 mph (7.6 km/h) from the southwest, and the pressure is 1002 mb. There is no precipitation expected today.

========== Query processing completed ========== 



This output shows an AI agent processing a user query about "latest news OpenShift."

* **📍 Step 1: InferenceStep**: The agent *infers* the need to search for information and decides to use the `brave_search` tool with the query "latest news OpenShift."
* **📍 Step 2: ToolExecutionStep**: The agent *executes* the `brave_search` tool. The JSON block is the *output* from this tool, providing search results with titles, URLs, snippets, and relevance scores.
* **📍 Step 3: InferenceStep**: The agent *processes* the search results from the tool and *infers* how to construct a final answer.
* **🤖 Model Response**: This is the AI agent's *final generated response* to the user's query, summarizing the key information extracted from the search results.

### Creating our first custom tool

This code defines a custom tool called `get_location` for agent to use

* **`@client_tool`**: This decorator from `llama_stack_client.lib.agents.client_tool` marks the `get_location` function as a tool that the agent can discover and use to fulfill user requests. It essentially registers this function within the agent's toolkit.

* **Docstring Notes**: The text within the triple quotes (`"""Docstring goes here"""`) serves as crucial instructions for the AI agent. It explains:
    * **What the tool does**: "Provides the user's location upon request."
    * **How it works**: Mentions using the `geocoder` library and determining location via IP address.
    * **Input parameters**: Describes the `query` parameter.
    * **Output**: Specifies what the tool returns (location information as a string).
    * **Example**: Shows how the tool can be used and its expected output.

The agent reads this docstring to understand the tool's purpose, how to call it (with what arguments), and what kind of result to expect. This allows the agent to strategically use the `get_location` tool when a user asks for their location.

> **Note:**
> This is just one way to create a custom tool, we can also import tools from libraries or use MCP Servers (Again, more on this later...) 
> 

In [7]:
from llama_stack_client.lib.agents.client_tool import client_tool

@client_tool
def get_location(query: str = "location"):
    """
    Provides the user's location upon request.

    This function uses the geocoder library to determine the user's location
    based on their IP address.  It returns the city, state, and country.

    :param query:  The query from the user.  Defaults to "location".
    :type query: str
    :return:  Information about the user's current location.
    :rtype: str

    Example:
        >>> get_location("where am i")
        "Your current location is: Some City, Some State, Some Country"
    """
    import geocoder
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

## Prompt Chaining (Optional)

Prompt chaining allows an AI agent to remember information across multiple turns within the same session. In this example, both user questions share the same `session_id`.

Because they share a session, the agent will remember the answer to the first question ("Where am I?") when interpreting the second question ("search for any weather-related risks in my area..."). This enables the agent to understand "my area" refers to the location identified in the previous turn, leading to a more context-aware and helpful response.

### Defining a Multi-tool Agent

This code initializes an AI agent with specific instructions and tools. 
The `tools` parameter is a list containing elements the agent can use. Here, it includes `get_location` (a custom function defined earlier) and `"builtin::websearch"` (a pre-built web searching capability in Llama Stack). 


The instructions (often refered to as "Prompt" guide the agent on when to use each of these tools based on the user's query.




In [8]:

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    # instructions="""You are a helpful assistant. 
    # When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    # """ ,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the brave_search tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    #sampling_params=sampling_params
)

### inspecting the Agent's response structure with Prompt Chaining

Notice that we are **NOT** generating a new session ID for each question, this way, the convesation memory is shared throughtout the session. 


In [9]:
user_prompts = [
    "Where am I?",
    "search for any weather-related risks in my area that could disrupt network connectivity or system availability?",
]

# Generate a new Unique Identifier for the session, but *NOT* for each question.

new_uuid = uuid.uuid4()
session_id = agent.create_session("prompt-chaining-session-{new_uuid}")

for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Columbus, Ohio, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
<|python_tag|>{"city": "Columbus", "state": "Ohio", "country": "US"}

========== Query processing completed ========== 


Processing user query: search for any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks near Columbus, Ohio, US'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks near Columbus, Ohio, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Columbus, Ohio, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Columbus', 'region': 'Ohio', 'country': 'United States of America', 'lat': 39.9611, 'lon': -82.9989, 'tz_id': 'America/New_York', 'localtime_epoch': 1747504099, 'localtime': '2025-05-17 13:48'}, 'current': {'last_updated_epoch': 1747503900, 'last_updated': '2025-05-17 13:45', 'temp_c': 22.8, 'temp_f': 73.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 17.0, 'wind_kph': 27.4, 'wind_degree': 265, 'wind_dir': 'W', 'pressure_mb': 1003.0, 'pressure_in': 29.62, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 44, 'cloud': 50, 'feelslike_c': 24.7, 'feelslike_f': 76.5, 'windchill_c': 21.3, 'windchill_f': 70.4, 'heatindex_c': 23.8, 'heatindex_f': 74.9, 'dewpoint_c': 11.6, 'dewpoint_f': 52.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 7.6, 'gust_mph': 20.4, 'gust_kph': 32.9}}",
│   │   │   'score': 0.982539,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus, Ohio Climate Change Risks and Hazards: Precipitation, Heat',
│   │   │   'url': 'https://climatecheck.com/ohio/columbus',
│   │   │   'content': 'Columbus, OHTop Climate Change Risks: Precipitation, Heat People in Columbus, OH are especially likely to experience increased risks from precipitation and heat. Precipitation and heat risk in Columbus, OH is very high. Precipitation risk in Columbus, OH The annual precipitation in Columbus, OH is projected to increase from about 36.7" to about 39.4".Extreme precipitation in any form can pose significant risks. Heat risk in Columbus, OH Property owners can check a specific address for flood risk including FEMA flood zone, then take steps to reduce their vulnerability to flooding damage.Climate change is increasing inland and coastal flooding risk due to sea level rise and increasing chances of extreme precipitation. The risks presented on this page reflect modeled averages for Columbus, OH under one projected emissions scenario and can vary for individual properties.',
│   │   │   'score': 0.7125728,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Damaging winds, hail, tornadoes possible ... - The Columbus Dispatch',
│   │   │   'url': 'https://www.dispatch.com/story/news/local/2025/05/15/columbus-weather-forecast-damaging-winds-hail-tornadoes-possible-thursday-friday-according-to-nws/83642723007/',
│   │   │   'content': 'Columbus only has a slight risk of severe weather Thursday and Friday, according to an NWS map. Here is what you can expect from the weather for the next week. Ohio heat wave getting worse: New',
│   │   │   'score': 0.6241337,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus Weather in 2025',
│   │   │   'url': 'https://www.extremeweatherwatch.com/cities/columbus/year-2025',
│   │   │   'content': "Columbus Weather in 2025 Columbus Weather in 2025 | Highest temperature | 80 °F | April 2 | | Lowest temperature | -3 °F | January 22 | What's the hottest temperature in Columbus so far this year? As of April 6, the highest temperature recorded in Columbus, Ohio in 2025 is 80 °F which happened on April 2. Highest Temperatures in Columbus in 2025 What's the coldest temperature in Columbus so far this year? As of April 6, the lowest temperature recorded in Columbus, Ohio in 2025 is -3 °F which happened on Jan 22. Lowest Temperatures in Columbus in 2025 January 2025 Columbus Weather February 2025 Columbus Weather March 2025 Columbus Weather April 2025 Columbus Weather",
│   │   │   'score': 0.54956865,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Enhanced Hazardous Weather Outlook Ohio - National Weather Service',
│   │   │   'url': 'https://www.weather.gov/iln/ehwooh',
│   │   │   'content'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, there are several weather-related risks in the Columbus, Ohio area that could disrupt network connectivity or system availability. These include:

* Precipitation risk: There is a high risk of precipitation in Columbus, Ohio, with an increase in annual precipitation projected from 36.7" to about 39.4". This can pose significant risks, including flooding damage.
* Heat risk: There is also a high risk of heat in Columbus, Ohio, with temperatures expected to rise due to climate change.
* Severe weather: There is a slight risk of severe weather, including damaging winds, hail, and tornadoes, on Thursday and Friday, according to the National Weather Service.

It is recommended to take steps to reduce vulnerability to flooding damage and secure unsecured property to prevent wind damage. It is also important to review weather safety tips and emergency kit preparations.

========== Query processing co

## Lab Summary: Interacting with Agents and Tools

In this lab, you took your first steps in understanding and interacting with AI agents using the Llama Stack framework and observe the benefits of Llama Stack in providing a structured and intuitive way to build intelligent agents with access to a variety of tools, both built-in and custom, and the ability to manage conversational memory for more coherent interactions. 

You learned how to:

* **Define a basic AI agent:** You created an agent and provided it with instructions on how to behave.
* **Utilize built-in tools:** You explored how agents can leverage pre-existing tools within Llama Stack, specifically the `websearch` tool, to gather information from the web.
* **Observe agent behavior:** You became familiar with the concepts of "Turns" and "Steps" that an agent goes through when processing a query, including the crucial "Tool Call" step where the agent decides to use a tool.
* **Create custom tools:** You experienced how to extend an agent's capabilities by defining your own specialized tool (`get_location`) using Python and decorators.
* **Implement prompt chaining:** You explored how to maintain context across multiple user interactions within the same session, allowing the agent to remember previous information and use it to answer subsequent questions more effectively.

Through these exercises, you gained a foundational understanding of how agents operate within the Llama Stack ecosystem and how they can be equipped with tools to perform specific tasks. 